# Data Loader

In [3]:
from imutils import paths
from pathlib import Path 
from matplotlib import pyplot as plt
import numpy as np
import argparse
import cv2
import os


from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
import glob
import os
import numpy as np
import cv2
import torch
from torchvision import transforms, utils
from PIL import Image

In [4]:
class SegDataset(Dataset):
    """Segmentation Dataset"""

    def __init__(
        self, 
        root_dir, 
        imageFolder, 
        maskFolder, 
        transform=None, 
        seed=None, 
        fraction=None, 
        subset=None, 
        imagecolormode='rgb', 
        maskcolormode='rgb'
    ):
        """
        Args:
            root_dir (string): Directory with all the images and should have the following structure.
            root
            --Images
            -----Img 1
            -----Img N
            --Mask
            -----Mask 1
            -----Mask N
            imageFolder (string) = 'Images' : Name of the folder which contains the Images.
            maskFolder (string)  = 'Masks : Name of the folder which contains the Masks.
            transform (callable, optional): Optional transform to be applied on a sample.
            seed: Specify a seed for the train and test split
            fraction: A float value from 0 to 1 which specifies the validation split fraction
            subset: 'Train' or 'Test' to select the appropriate set.
            imagecolormode: 'rgb' or 'grayscale'
            maskcolormode: 'rgb' or 'grayscale'
        """
        self.color_dict = {'rgb': 1, 'grayscale': 0}
        assert(imagecolormode in ['rgb', 'grayscale'])
        assert(maskcolormode in ['rgb', 'grayscale'])

        self.imagecolorflag = self.color_dict[imagecolormode]
        self.maskcolorflag = self.color_dict[maskcolormode]
        self.root_dir = root_dir
        self.transform = transform
        if not fraction:
            self.image_names = sorted(
                glob.glob(os.path.join(self.root_dir, imageFolder, '*')))
            self.mask_names = sorted(
                glob.glob(os.path.join(self.root_dir, maskFolder, '*')))
        else:
            assert(subset in ['Train', 'Test'])
            self.fraction = fraction
            self.image_list = np.array(
                sorted(glob.glob(os.path.join(self.root_dir, imageFolder, '*'))))
            self.mask_list = np.array(
                sorted(glob.glob(os.path.join(self.root_dir, maskFolder, '*'))))
            if seed:
                np.random.seed(seed)
                indices = np.arange(len(self.image_list))
                np.random.shuffle(indices)
                self.image_list = self.image_list[indices]
                self.mask_list = self.mask_list[indices]
            if subset == 'Train':
                self.image_names = self.image_list[:int(
                    np.ceil(len(self.image_list)*(1-self.fraction)))]
                self.mask_names = self.mask_list[:int(
                    np.ceil(len(self.mask_list)*(1-self.fraction)))]
            else:
                self.image_names = self.image_list[int(
                    np.ceil(len(self.image_list)*(1-self.fraction))):]
                self.mask_names = self.mask_list[int(
                    np.ceil(len(self.mask_list)*(1-self.fraction))):]

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        if self.imagecolorflag:
            image = cv2.imread(
                img_name, self.imagecolorflag)
        else:
            image = cv2.imread(img_name, self.imagecolorflag)
        
        msk_name = self.mask_names[idx]
        if self.maskcolorflag:
            mask = cv2.imread(msk_name, self.maskcolorflag)
        else:
            mask = cv2.imread(msk_name, self.maskcolorflag)
        

        #image = Image.open(img_name)
        #mask = Image.open()
     
        sample = {'image': image, 'mask': mask}

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return {'image': image, 'mask': mask}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample, maskresize=None, imageresize=None):
        image, mask = sample['image'], sample['mask']
        if len(mask.shape) == 2:
            mask = mask.reshape((1,)+mask.shape)
        if len(image.shape) == 2:
            image = image.reshape((1,)+image.shape)
        return {'image': torch.from_numpy(image),
                'mask': torch.from_numpy(mask)}


class Normalize(object):
    """Normalize image"""

    def __call__(self, sample):
        image, mask = sample['image'], sample['mask']
        return {'image': image.type(torch.FloatTensor)/255,
                'mask': mask.type(torch.FloatTensor)/255}


def get_dataloader_single_folder(data_dir, imageFolder='Images', maskFolder='Masks', fraction=0.2, batch_size=4):
    """Create training and testing dataloaders from a single folder."""
    data_transforms = {
        'Train': transforms.Compose([
            transforms.ToPILImage(), 
            Resize((256, 256)), 
            transforms.ToTensor(), 
            transforms.Normalize(0,1)
        ]),
        'Test': transforms.Compose([
            transforms.ToPILImage(), 
            Resize((256, 256)), 
            transforms.ToTensor(), 
            transforms.Normalize(0,1)
        ]),
    }

    image_datasets = {
        x: SegDataset(
            data_dir, 
            imageFolder=imageFolder, 
            maskFolder=maskFolder, 
            seed=100, 
            fraction=fraction, 
            subset=x, 
            transform=data_transforms[x]
        ) for x in ['Train', 'Test']
    }
    dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size,
                                 shuffle=True, num_workers=8)
                   for x in ['Train', 'Test']}
    return dataloaders

In [3]:
dataloaders = get_dataloader_single_folder(
    data_dir="../data/finetune", 
    imageFolder='images', 
    maskFolder='masks', 
    fraction=0.2, 
    batch_size=4,
)

# Model Building

In [4]:
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

In [5]:
def createDeepLabv3(outputchannels=1):
    model = models.segmentation.deeplabv3_resnet101(
        pretrained=True, progress=True)
    # Added a Tanh activation after the last convolution layer
    model.classifier = DeepLabHead(2048, outputchannels)
    # Set the model in training mode
    model.train()
    return model


model = createDeepLabv3(3)

/home/adam13/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/adam13/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Finetune

In [6]:
import csv
import copy
import time
from tqdm import tqdm
import torch
import numpy as np
import os

from sklearn.metrics import f1_score, roc_auc_score


In [7]:
def train_model(model, criterion, dataloaders, optimizer, metrics, bpath, num_epochs=3):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    # Use gpu if available
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Initialize the log file for training and testing loss and metrics
    fieldnames = ['epoch', 'Train_loss', 'Test_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Test_{m}' for m in metrics.keys()]
    with open(os.path.join(bpath, 'log.csv'), 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)
        # Each epoch has a training and validation phase
        # Initialize batch summary
        batchsummary = {a: [0] for a in fieldnames}

        for phase in ['Train', 'Test']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            # Iterate over data.
            for sample in tqdm(iter(dataloaders[phase])):
                inputs = sample['image'].to(device)
                masks = sample['mask'].to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    loss = criterion(outputs['out'], masks)
                    y_pred = outputs['out'].data.cpu().numpy().ravel()
                    y_true = masks.data.cpu().numpy().ravel()
                    for name, metric in metrics.items():
                        if name == 'f1_score':
                            # Use a classification threshold of 0.1
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true > 0, y_pred > 0.1))
                        else:
                            batchsummary[f'{phase}_{name}'].append(
                                metric(y_true.astype('uint8'), y_pred))

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()
            batchsummary['epoch'] = epoch
            epoch_loss = loss
            batchsummary[f'{phase}_loss'] = epoch_loss.item()
            print('{} Loss: {:.4f}'.format(
                phase, loss))
        for field in fieldnames[3:]:
            batchsummary[field] = np.mean(batchsummary[field])
        print(batchsummary)
        with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writerow(batchsummary)
            # deep copy the model
            if phase == 'Test' and loss < best_loss:
                best_loss = loss
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Lowest Loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [8]:
epochs = 25
bpath = "../exp/"

# Specify the loss function
criterion = torch.nn.MSELoss(reduction='mean')
# Specify the optimizer with a lower learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Specify the evalutation metrics
metrics = {'f1_score': f1_score, 'auroc': roc_auc_score}

trained_model = train_model(model, criterion, dataloaders,
                            optimizer, bpath=bpath, metrics=metrics, num_epochs=epochs)

torch.save(trained_model, os.path.join(bpath, 'weights.pt'))

Epoch 1/25
----------


100%|██████████| 653/653 [05:01<00:00,  2.17it/s]

Train Loss: 0.0636



100%|██████████| 163/163 [00:57<00:00,  2.86it/s]

Test Loss: 0.0449
{'epoch': 1, 'Train_loss': 0.06356216967105865, 'Test_loss': 0.04493154212832451, 'Train_f1_score': 0.7226524072023118, 'Train_auroc': 0.9257491828273825, 'Test_f1_score': 0.8184593609955575, 'Test_auroc': 0.9484445226694365}
Epoch 2/25
----------


...

Test Loss: 0.0402
{'epoch': 24, 'Train_loss': 0.009436545893549919, 'Test_loss': 0.04023609682917595, 'Train_f1_score': 0.9395694362440709, 'Train_auroc': 0.9879775307124231, 'Test_f1_score': 0.8506346967364874, 'Test_auroc': 0.959732659634605}
Epoch 25/25
----------



100%|██████████| 653/653 [04:58<00:00,  2.19it/s]

Train Loss: 0.0144



100%|██████████| 163/163 [00:59<00:00,  2.76it/s]


Test Loss: 0.0331
{'epoch': 25, 'Train_loss': 0.014367426745593548, 'Test_loss': 0.03307809680700302, 'Train_f1_score': 0.9063223533306467, 'Train_auroc': 0.9784981711019782, 'Test_f1_score': 0.8401123609025737, 'Test_auroc': 0.9646521096671403}
Training complete in 544m 3s
Lowest Loss: 0.008766
